<header style="padding:1px;background:#f9f9f9;border-top:3px solid #00b2b1"><img id="Teradata-logo" src="https://www.teradata.com/Teradata/Images/Rebrand/Teradata_logo-two_color.png" alt="Teradata" width="220" align="right" />

<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Price prediction of house using UDF</b>
</header>

### Disclaimer
The sample code (“Sample Code”) provided is not covered by any Teradata agreements. Please be aware that Teradata has no control over the model responses to such sample code and such response may vary. The use of the model by Teradata is strictly for demonstration purposes and does not constitute any form of certification or endorsement. The sample code is provided “AS IS” and any express or implied warranties, including the implied warranties of merchantability and fitness for a particular purpose, are disclaimed. In no event shall Teradata be liable for any direct, indirect, incidental, special, exemplary, or consequential damages (including, but not limited to, procurement of substitute goods or services; loss of use, data, or profits; or business interruption) sustained by you or a third party, however caused and on any theory of liability, whether in contract, strict liability, or tort arising in any way out of the use of this sample code, even if advised of the possibility of such damage.

<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Usecase</b><br>
Create a user defined function to predict prices of residential property based on various features using the pre-trained GLM model on the housing price dataset.

<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Import the required libraries</b>

In [2]:
# Importing required libraries
import getpass
from teradataml import *
from teradatasqlalchemy.types import FLOAT
from teradataml.dataframe.functions import udf

<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Connect to Vantage</b>

In [3]:
# Create the connection.
td_context = create_context(host=getpass.getpass("Hostname: "), username=getpass.getpass("Username: "), password=getpass.getpass("Password: "))

Hostname:  ········
Username:  ········
Password:  ········


In [4]:
# Set the Authentication Token
set_config_params(auth_token=getpass.getpass("auth_data: "), ues_url=getpass.getpass("ues_url: "))

auth_data:  ········
ues_url:  ········


True

<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Loading Dataset</b>

In [5]:
# Load the example dataset.
load_example_data("GLMPredict", ["housing_test","housing_train"])
train = DataFrame('housing_train')
test = DataFrame('housing_test')

In [6]:
# Housing train dataset
train.head()

sn,price,lotsize,bedrooms,bathrms,stories,driveway,recroom,fullbase,gashw,airco,garagepl,prefarea,homestyle
3,49500.0,3060.0,3,1,1,yes,no,no,no,no,0,no,Classic
5,61000.0,6360.0,2,1,1,yes,no,no,no,no,0,no,Eclectic
6,66000.0,4160.0,3,1,1,yes,yes,yes,no,yes,0,no,Eclectic
7,66000.0,3880.0,3,2,2,yes,no,yes,no,no,2,no,Eclectic
9,83800.0,4800.0,3,1,1,yes,yes,yes,no,no,0,no,Eclectic
10,88500.0,5500.0,3,2,4,yes,yes,no,no,yes,1,no,Eclectic
8,69000.0,4160.0,3,1,3,yes,no,no,no,no,0,no,Eclectic
4,60500.0,6650.0,3,1,2,yes,yes,no,no,no,0,no,Eclectic
2,38500.0,4000.0,2,1,1,yes,no,no,no,no,0,no,Classic
1,42000.0,5850.0,3,1,2,yes,no,yes,no,no,1,no,Classic


In [7]:
# Housing test dataset
test.head()

sn,price,lotsize,bedrooms,bathrms,stories,driveway,recroom,fullbase,gashw,airco,garagepl,prefarea,homestyle
25,42000.0,4960.0,2,1,1,yes,no,no,no,no,0,no,Classic
53,68000.0,9166.0,2,1,1,yes,no,yes,no,yes,2,no,Eclectic
111,43000.0,5076.0,3,1,1,no,no,no,no,no,0,no,Classic
117,93000.0,3760.0,3,1,2,yes,no,no,yes,no,2,no,Eclectic
140,43000.0,3750.0,3,1,2,yes,no,no,no,no,0,no,Classic
142,40000.0,2650.0,3,1,2,yes,no,yes,no,no,1,no,Classic
132,44500.0,3850.0,3,1,2,yes,no,no,no,no,0,no,Classic
38,67000.0,5170.0,3,1,4,yes,no,no,no,yes,0,no,Eclectic
16,37900.0,3185.0,2,1,1,yes,no,no,no,yes,0,no,Classic
13,27000.0,1700.0,3,1,2,yes,no,no,no,no,0,no,Classic


<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Setting up the Environment</b>

In [8]:
list_user_envs()

,env_name,env_description,base_env_name,language,conda
0,conda_env_3_10_demo,Conda environment for notebook demo,python_3.10,python,True
1,demo_env,Demo env 1.,python_3.10,Python,False
2,non_conda_env_3_8_demo,Non Conda environment for notebook demo,python_3.8,Python,False
3,openml_env,DONT DELETE: OpenML environment,python_3.10,Python,False
4,openml_env_dhan,DONT DELETE: OpenML environment,python_3.10,Python,False
5,testenv,This env 'testenv' is created with base env 'p...,python_3.10,Python,False


In [9]:
# create a new environment use create_env function, if environment is already created use get_env to get access to the environment.
env = create_env('ml_env', 'python_3.10', 'Test environment for UDF')

User environment 'ml_env' created.


In [10]:
# Install the required packages to run the user defined function in the user environment using install_lib function Of UserEnv class.
env.install_lib(['pandas', 'statsmodels'])

,Claim Id,File/Libs/Model,Method Name,Stage,Timestamp,Additional Details
0,b5003d92-5550-4c89-b7e7-1921f349fb52,"pandas, statsmodels",install_lib,Started,2024-09-04T12:08:03Z,
1,b5003d92-5550-4c89-b7e7-1921f349fb52,"pandas, statsmodels",install_lib,Finished,2024-09-04T12:11:04Z,


In [11]:
env.libs

,name,version
0,numpy,2.1.1
1,packaging,24.1
2,pandas,2.2.2
3,patsy,0.5.6
4,pip,23.0.1
5,python-dateutil,2.9.0.post0
6,pytz,2024.1
7,scipy,1.14.1
8,setuptools,65.5.0
9,six,1.16.0


<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Fitting the GLM model</b>

In [13]:
# Import all required modules.
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [14]:
# Fit the model on train dataset using R-style formula to specify categorical variables as well.
model = smf.glm('price ~ C(recroom) + lotsize + stories + garagepl + C(gashw) +'
                ' bedrooms + C(driveway) + C(airco) + C(homestyle) + bathrms +'
                ' C(fullbase) + C(prefarea)',
                family=sm.families.Gaussian(), data=train.to_pandas()).fit()

<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Uploading the model in the User Environment</b>

In [15]:
# Save the model.
import pickle
with open("udf_model.pkl", "wb") as f:
    pickle.dump(model, f)

In [16]:
# Install the model object file on the user environment.
env.install_file(file_path='udf_model.pkl', replace = True)

File 'udf_model.pkl' replaced successfully in the remote user environment 'ml_env'.


True

<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Create a UDF and run it on Vantage to predict the house price.</b>

In [17]:
@udf(env_name = "ml_env", returns = FLOAT())
def predict_house_price(lotsize, bedrooms, bathrms, stories, driveway, recroom, fullbase, gashw, airco, garagepl, prefarea, homestyle):
    import pickle
    import pandas as pd
    import statsmodels.api as sm
    import statsmodels.formula.api as smf

    # Load the glm model.
    with open("udf_model.pkl", 'rb') as f:
        model = pickle.load(f)
        
    # Create a pandas dataframe of the row values as predict method accepts pandas dataframe as input.
    data = [{'lotsize': lotsize, 'bedrooms': bedrooms, 'bathrms': bathrms, 'stories': stories, 'driveway': driveway, 'recroom': recroom, 'fullbase': fullbase, 'gashw': gashw,'airco': airco, 'garagepl': garagepl,'prefarea': prefarea,'homestyle':homestyle}]  
    df = pd.DataFrame(data)
    price = list(model.predict(df))[0]
    return price   

In [18]:
#Assign the Column Expression returned by user defined function to the DataFrame.
out_df = test.assign(prediction = predict_house_price('lotsize', 'bedrooms', 'bathrms', 'stories', 'driveway', 'recroom', 'fullbase', 'gashw', 'airco', 'garagepl', 'prefarea', 'homestyle'))

# Print the result.
out_df

sn,price,lotsize,bedrooms,bathrms,stories,driveway,recroom,fullbase,gashw,airco,garagepl,prefarea,homestyle,prediction
403,77500.0,6825.0,3,1,1,yes,yes,yes,no,yes,0,yes,Eclectic,79857.27570249245
237,43000.0,3630.0,4,1,2,yes,no,no,no,no,3,no,Classic,46593.47360830345
440,69000.0,6862.0,3,1,2,yes,no,no,no,yes,2,yes,Eclectic,80549.2695156874
355,86900.0,4300.0,6,2,2,yes,no,no,no,no,0,no,Eclectic,68425.81252036957
472,60500.0,2787.0,3,1,1,yes,no,yes,no,no,0,yes,Eclectic,63767.070382294914
255,61000.0,4360.0,4,1,2,yes,no,no,no,no,0,no,Eclectic,60214.17611659672
540,85000.0,7320.0,4,2,2,yes,no,no,no,no,0,no,Eclectic,73146.10118254229
340,62500.0,3900.0,3,1,2,yes,no,no,no,no,0,no,Eclectic,58474.84506181388
364,72000.0,10700.0,3,1,2,yes,yes,yes,no,no,0,no,Eclectic,77422.14880877169
459,44555.0,2398.0,3,1,1,yes,no,no,no,no,0,yes,Classic,42818.35762498098


<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Disconnect from Vantage</b>

In [19]:
remove_context()

True